In [1]:
"""
Generate a master key file, mapping each LINE_NM to a PROFILE value based on majority rules 
    from the StandardsData.xlsx file. This resulting file just has a header, and each line in
    the file is a different LINE_NM to PROFILE mapping. There may be multiple duplicate LINE_NM keys, 
    that map to various different profiles. We determine the profiles by choosing all profiles 
    that LINE_NM maps to, for which it maps to them >= 25% of it's mappings.
    
-Blake Edwards / Dark Element
"""
import pandas as pd
import numpy as np
from base import *
import math


#Read in file, with all columns except for PROJECT_NM, DNA_PLATE, PLANT_ID
#We keep rows with empty entries, since they may have some valid entries.
data = pd.read_excel("StandardsData.xlsx")
data = data.drop(["PROJECT_NM", "DNA_PLATE", "PLANT_ID"], axis=1)
data = data.rename({"LINES":"LINE_NM"}, axis=1)
data

,LINE_NM,11_10030,11_10041,11_10129,11_10186,11_10400,11_10422,11_10621,11_11105,11_11302,...,11_21400,12_10579,12_30331,12_30427,12_30521,12_30743,12_30956,12_31350,12_31382,12_31428
0,BRS BRAU,BB,BB,BB,AA,AA,BB,AA,AA,BB,...,NaN,BB,AA,AA,AA,AA,AA,AA,BB,AA
1,BRS BRAU,BB,BB,BB,AA,AA,BB,NaN,AA,NaN,...,NaN,NaN,AA,NaN,AA,NaN,AA,NaN,NaN,NaN
2,BRS BRAU,BB,BB,BB,AA,AA,BB,AA,AA,Shortfall,...,BB,BB,AA,AA,AA,AA,AA,AA,BB,AA
3,BRS BRAU,BB,BB,Shortfall,AA,AA,BB,AA,AA,BB,...,BB,Shortfall,AA,AA,AA,AA,AA,AA,BB,Shortfall
4,BRS BRAU,BB,BB,BB,AA,AA,BB,AA,AA,Shortfall,...,BB,BB,AA,AA,AA,AA,AA,AA,BB,AA
5,BRS BRAU,BB,BB,BB,AA,AA,BB,AA,AA,BB,...,BB,BB,AA,AA,AA,AA,AA,AA,BB,AA
6,BRS BRAU,BB,BB,BB,AA,AA,BB,AA,AA,BB,...,BB,BB,AA,AA,AA,AA,AA,AA,BB,AA
7,BRS BRAU,AA,BB,AA,AA,AA,BB,AA,AA,BB,...,BB,AA,AA,AA,BB,AA,AA,AA,BB,AA
8,BRS BRAU,BB,BB,BB,AA,AA,BB,AA,AA,BB,...,BB,BB,AA,AA,AA,AA,AA,AA,BB,AA
9,BRS CAUE,AA,AA,AA,BB,AA,BB,AA,AA,BB,...,NaN,BB,BB,BB,BB,AA,AA,AA,AA,BB


In [2]:
#Get lists of all marker_nm and line_nm values
marker_nms = list(data.columns[1:])
line_nms = np.unique(data["LINE_NM"])

"""
We now get the number of occurrences of each valid call value, 
    for each marker_nm value,
    for each line_nm value,
    throughout the entire file.
    
We use a multi-tiered dictionary of the form marker_nm -> line_nm -> call -> count,
    since using the form line_nm -> marker_nm -> call -> count would result in a large amount
    of duplicate values. 
    
We'll then use this to find all line_nm: profile mappings. 
"""
#Create these ahead of time to avoid checks in the loop, initialize our multi-tiered dictionary
profiles = {}
for marker_nm in marker_nms:
    profiles[marker_nm] = {}
    for line_nm in line_nms:
        profiles[marker_nm][line_nm] = {}

for i, row in data.iterrows():
    for marker_nm in marker_nms:
        #If this profile is not valid, move on to the next one.
        profile = row[marker_nm]
        
        if not profile_is_valid(profile):
            continue

        #Otherwise, increment this profile's count in it's respective line_nm and marker_nm entry 
        #in our profiles dictionary, setting its count to 1 if it doesn't already exist.
        line_nm = row["LINE_NM"]    
        if profile in profiles[marker_nm][line_nm]:
            profiles[marker_nm][line_nm][profile] += 1
        else:
            profiles[marker_nm][line_nm][profile] = 1
profiles

{'11_10030': {'ANAG 01': {'AA': 1, 'BB': 23},
  'BRS BRAU': {'AA': 1, 'BB': 8},
  'BRS CAUE': {'AA': 16},
  'BRS KORBEL': {'BB': 7},
  'BRS QUARANTA': {'AA': 6},
  'HARRINGTON': {'AA': 60},
  'Merit': {'AA': 62},
  'STELLAR-ND': {'AA': 16},
  'Stander': {'BB': 14}},
 '11_10041': {'ANAG 01': {'AA': 23, 'BB': 1},
  'BRS BRAU': {'BB': 9},
  'BRS CAUE': {'AA': 16},
  'BRS KORBEL': {'BB': 7},
  'BRS QUARANTA': {'AA': 6},
  'HARRINGTON': {'AB': 45, 'BB': 16},
  'Merit': {'AA': 1, 'BB': 60},
  'STELLAR-ND': {'AA': 16},
  'Stander': {'AA': 14}},
 '11_10129': {'ANAG 01': {},
  'BRS BRAU': {'AA': 1, 'BB': 7},
  'BRS CAUE': {'AA': 17, 'BB': 1},
  'BRS KORBEL': {'AA': 7},
  'BRS QUARANTA': {'AA': 6, 'AB': 1},
  'HARRINGTON': {'BB': 61},
  'Merit': {'AA': 60, 'BB': 2},
  'STELLAR-ND': {'AA': 15},
  'Stander': {'AB': 1, 'BB': 14}},
 '11_10186': {'ANAG 01': {'AA': 24},
  'BRS BRAU': {'AA': 9},
  'BRS CAUE': {'BB': 16},
  'BRS KORBEL': {'BB': 7},
  'BRS QUARANTA': {'BB': 6},
  'HARRINGTON': {'BB': 61}

In [3]:
"""
Go through now complete profiles dictionary for each line_nm entry, and for every marker_nm
    entry, remove all profiles which make up < 25% of the total entries for this marker_nm for this line_nm.
"""
for marker_nm in marker_nms:
    for line_nm in line_nms:
        #Get sum number of profile occurences for this line_nm and marker_nm
        n = sum(profiles[marker_nm][line_nm].values())
        
        #Loop through profiles and remove every one that makes up <25% of the total entries
        for profile in list(profiles[marker_nm][line_nm].keys()):
            if profiles[marker_nm][line_nm][profile]/n < .25:
                del profiles[marker_nm][line_nm][profile]
        
    
profiles

{'11_10030': {'ANAG 01': {'BB': 23},
  'BRS BRAU': {'BB': 8},
  'BRS CAUE': {'AA': 16},
  'BRS KORBEL': {'BB': 7},
  'BRS QUARANTA': {'AA': 6},
  'HARRINGTON': {'AA': 60},
  'Merit': {'AA': 62},
  'STELLAR-ND': {'AA': 16},
  'Stander': {'BB': 14}},
 '11_10041': {'ANAG 01': {'AA': 23},
  'BRS BRAU': {'BB': 9},
  'BRS CAUE': {'AA': 16},
  'BRS KORBEL': {'BB': 7},
  'BRS QUARANTA': {'AA': 6},
  'HARRINGTON': {'AB': 45, 'BB': 16},
  'Merit': {'BB': 60},
  'STELLAR-ND': {'AA': 16},
  'Stander': {'AA': 14}},
 '11_10129': {'ANAG 01': {},
  'BRS BRAU': {'BB': 7},
  'BRS CAUE': {'AA': 17},
  'BRS KORBEL': {'AA': 7},
  'BRS QUARANTA': {'AA': 6},
  'HARRINGTON': {'BB': 61},
  'Merit': {'AA': 60},
  'STELLAR-ND': {'AA': 15},
  'Stander': {'BB': 14}},
 '11_10186': {'ANAG 01': {'AA': 24},
  'BRS BRAU': {'AA': 9},
  'BRS CAUE': {'BB': 16},
  'BRS KORBEL': {'BB': 7},
  'BRS QUARANTA': {'BB': 6},
  'HARRINGTON': {'BB': 61},
  'Merit': {'BB': 72},
  'STELLAR-ND': {'AA': 18},
  'Stander': {'BB': 15}},
 '

In [4]:
"""
Use our now complete profiles dictionary to create an nx2 profile_map list for each pair mapping
    LINE_NM to PROFILE values. We additionally create multiple entries for a LINE_NM 
    in the case of multiple possible PROFILE combinations.
    
We use a list so that we can more efficiently append and sort, and since there may
    be both duplicate LINE_NMs and duplicate PROFILEs (but not both at once) throughout the list,
    and attempting to use a dictionary for this would be difficult.
"""
profile_map = []
for line_nm in line_nms:
    #To be used to create all possible profile combinations for this line_nm
    combinations = []
    for marker_nm in marker_nms:
        #Add a list of all profiles this line_nm maps to with this marker_nm
        combinations.append(list(profiles[marker_nm][line_nm].keys()))
    
    print(line_nm, combinations)
    
    #Add all profile combinations for this line_nm to the main profile_map
    for combination in get_profile_combinations(combinations):
        profile_map.append([line_nm, combination])
        
profile_map

ANAG 01 [['BB'], ['AA'], [], ['AA'], [], [], ['BB'], [], [], [], [], [], [], [], [], [], [], [], ['AA'], [], [], [], [], ['BB'], ['AA'], [], [], [], [], [], [], [], [], [], ['BB']]
BRS BRAU [['BB'], ['BB'], ['BB'], ['AA'], ['AA'], ['BB'], ['AA'], ['AA'], ['BB'], ['AA'], ['BB'], ['BB'], ['BB'], ['AA'], ['AA'], ['AA'], ['BB'], ['BB'], ['BB'], ['AA'], ['AA'], ['BB'], ['BB'], ['AA'], ['AA'], ['BB'], ['BB'], ['AA'], ['AA'], ['AA'], ['AA'], ['AA'], ['AA'], ['BB'], ['AA']]
BRS CAUE [['AA'], ['AA'], ['AA'], ['BB'], ['AA'], ['BB'], ['AA'], ['AA'], ['BB'], ['BB'], ['AA'], ['BB'], ['BB'], ['BB'], ['AA'], ['AA'], ['AA'], ['BB'], ['BB'], ['AA'], ['AA'], ['BB'], ['AA'], ['BB'], ['AA'], ['AA'], ['BB'], ['BB'], ['BB'], ['AA', 'BB'], ['AA'], ['AA'], ['AA'], ['AA'], ['BB']]
BRS KORBEL [['BB'], ['BB'], ['AA'], ['BB'], ['AA'], ['BB'], ['BB'], ['AA'], ['BB'], ['AA'], ['AA'], ['BB'], ['BB'], ['AA'], ['AA'], ['AA'], ['AA'], ['BB'], ['AA'], ['AA'], ['AA'], ['BB'], ['AA'], ['BB'], ['BB'], ['BB'], ['BB'], ['AA'

[['ANAG 01',
  ['BB',
   'AA',
   '',
   'AA',
   '',
   '',
   'BB',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   'AA',
   '',
   '',
   '',
   '',
   'BB',
   'AA',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   'BB']],
 ['BRS BRAU',
  ['BB',
   'BB',
   'BB',
   'AA',
   'AA',
   'BB',
   'AA',
   'AA',
   'BB',
   'AA',
   'BB',
   'BB',
   'BB',
   'AA',
   'AA',
   'AA',
   'BB',
   'BB',
   'BB',
   'AA',
   'AA',
   'BB',
   'BB',
   'AA',
   'AA',
   'BB',
   'BB',
   'AA',
   'AA',
   'AA',
   'AA',
   'AA',
   'AA',
   'BB',
   'AA']],
 ['BRS CAUE',
  ['AA',
   'AA',
   'AA',
   'BB',
   'AA',
   'BB',
   'AA',
   'AA',
   'BB',
   'BB',
   'AA',
   'BB',
   'BB',
   'BB',
   'AA',
   'AA',
   'AA',
   'BB',
   'BB',
   'AA',
   'AA',
   'BB',
   'AA',
   'BB',
   'AA',
   'AA',
   'BB',
   'BB',
   'BB',
   'AA',
   'AA',
   'AA',
   'AA',
   'AA',
   'BB']],
 ['BRS CAUE',
  ['AA',
   'AA',
   'AA',
   'BB',
   'AA',
   

In [5]:
"""
At this point profile_map is our completed list, mapping each line_nm to its profile combination(s). 
    The profile combinations are represented as lists of strings so that we can still reference marker_nm,
    and since some entries may be empty.

We use this to create our final key dataframe and CSV.
"""

#Initialize this ahead of time since we know the size
data = pd.DataFrame("", index=np.arange(len(profile_map)), columns=["LINE_NM"]+marker_nms)

#Fill with our values
for i, row in enumerate(profile_map):
    data.iat[i, 0] = row[0]
    for j, marker_nm in enumerate(row[1]):
        data.iat[i,j+1] = marker_nm

#Make CSV
data.to_csv("key.csv", index=False)